In [17]:
import pandas as pd
import nltk
import re
import numpy as np

stopwords = set(nltk.corpus.stopwords.words("english"))

In [18]:
df = pd.read_excel("Dataset B1.xlsx")
df.head()

,text,type
0,@ACNI2012 @TheToka920 Never knew having 1 or 2...,sports
1,"MYCA Magical Moments:\n\nSeptember, 2011: Sham...",sports
2,The current state of last year's @BBL finalist...,sports
3,@HOLLYJISOO Why did you bring a cricket...,sports
4,Babar Azam only Pakistani included in the ICC ...,sports


In [23]:
# clean data: remove weird characters and stopwords
for idx, row in df.iterrows():
    row['text'] = " ".join([w.lower() for w in re.findall(r"\w+", row['text']) if w.lower() not in stopwords])
df.head()

,text,type
0,acni2012 thetoka920 never knew 1 2 followers a...,sports
1,myca magical moments september 2011 sham choto...,sports
2,current state last year bbl finalists starsbbl...,sports
3,hollyjisoo bring cricket,sports
4,babar azam pakistani included icc odi team yea...,sports


In [33]:
def naive_bayes_text_classification(data, target_col, sentence, k):
    targets = list(np.unique(data[target_col]))
    words = sentence.split()
    word_counts = {t: {w: 0 for w in words} for t in targets}
    total_counts = {t: 0 for t in targets}

    V = set()
    for _, row in data.iterrows():
        cur_words = row['text'].split()
        total_counts[row[target_col]] += len(cur_words)
        for w in cur_words:
            V.add(w)
            if w in word_counts[row[target_col]]:
                word_counts[row[target_col]][w] += 1
    
    min_p_class, t_class = -np.inf, None
    for t in targets:
        p = len(data[data[target_col] == t])/len(data)
        for w, c in word_counts[t].items():
            p *= (c + k)/(total_counts[t] + k*len(V))
        if p >= min_p_class:
            min_p_class, t_class = p, t

    return t_class

In [34]:
test_sentence = "in a recent election, gandhi emerged victorious"
targ = naive_bayes_text_classification(df, "type", test_sentence, 1)
targ

'sports'

In [35]:
test_sentence = "king kohli dominated the match today"
targ = naive_bayes_text_classification(df, "type", test_sentence, 1)
targ

'sports'